# Prognozowanie szeregów czasowych

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Naszą główną biblioteką do szeregów czasowych będzie [sktime](https://www.sktime.net/en/stable/index.html), oferujące interfejs podobny do Scikit-learn. Jest to wrapper na wiele innych bibliotek, między innymi:
- [statsforecast](https://github.com/Nixtla/statsforecast) - wydajna implementacja wielu metod prognozowania, m.in. AutoARIMA i AutoETS
- [pmdarima](https://alkaline-ml.com/pmdarima/) - testy statystyczne dla szeregów czasowych oraz alternatywna implementacja AutoARIMA
- [statsmodels](https://www.statsmodels.org/stable/index.html) - niektóre metody dekompozycji szeregów czasowych i prognozowania

Do obliczania testów statystycznych wykorzystamy [scipy](https://docs.scipy.org/doc/scipy/index.html) oraz [statsmodels](https://www.statsmodels.org/stable/index.html).

## Prognozowanie inflacji w Polsce

Problem przewidywania inflacji (wskaźników cen towarów i usług konsumpcyjnych) jest bardzo powszechny, i wykonuje go zasadniczo każde państwo i większa instytucja finansowa. W praktyce jest to cała grupa interesujących problemów, bo mamy inflację, inflację bazową (po wyłączeniu najbardziej zmiennych czynników, np. cen żywności), 

W Polsce podstawowe dane o inflacji [publikuje Główny Urząd Statystyczny (GUS)](https://stat.gov.pl/obszary-tematyczne/ceny-handel/wskazniki-cen/wskazniki-cen-towarow-i-uslug-konsumpcyjnych-pot-inflacja-/miesieczne-wskazniki-cen-towarow-i-uslug-konsumpcyjnych-od-1982-roku/), z częstotliwością miesięczną, kwartalną, półroczną i roczną. Bardziej szczegółowe informacje publikują inne instytucje, bo zależą od przyjętej metodyki, np. inflację bazową [oblicza i publikuje Narodowy Bank Polski (NBP)](https://nbp.pl/statystyka-i-sprawozdawczosc/inflacja-bazowa/).

Prognozowanie inflacji jest wyzwaniem, bo typowo inflacja:
- ma wyraźne cykle, ale wysoce nieregularne
- jest implicite związana z wieloma czynnikami zewnętrznymi (gospodarka światowa, decyzje polityczne etc.)
- nie ma wyraźnej sezonowości
- interesuje nas prognozowanie na wielu poziomach szczególowości, np. miesięczne, kwartalne, roczne

Zajmiemy się danymi z GUS, o częstotliwości miesięcznej. Aby otrzymać inflację rok do roku (RDR), z którą typowo mamy do czynienia, trzeba odjąć 100 od podawanych wartości.

In [ ]:
df = pd.read_csv("polish_inflation.csv")
df = df.rename(columns={"Rok": "year", "Miesiąc": "month", "Wartość": "value"})

# create proper date column
df["day"] = 1
df["date"] = pd.to_datetime(df[["year", "month", "day"]])
df["date"] = df["date"].dt.to_period("M")

# set datetime index
df = df.set_index(df["date"], drop=True)
df = df.sort_index()

# leave only time series values
df = df["value"] - 100
df

Do rysowania szeregów czasowych najłatwiej użyć funkcji `plot_series` z sktime, która ustawi nam od razu ładnie opisy na osi X.

In [ ]:
from sktime.utils.plotting import plot_series

plot_series(df, title="Polish inflation")

Tu nie ma błędu - lata 90 były ciekawym okresem, z [hiperinflacją](https://pl.wikipedia.org/wiki/Hiperinflacja#Polska_%E2%80%93_lata_80._XX_wieku), późniejszą [terapią szokową](https://en.wikipedia.org/wiki/Shock_therapy_(economics)) i realizacją [planu Balcerowicza](https://pl.wikipedia.org/wiki/Plan_Balcerowicza). Z perspektywy prognozowania szeregów czasowych jest to ewidentny outlier, ale przy tym bardzo długi. W związku z tym ograniczymy się do późniejszego okresu, od roku 2000.

In [ ]:
df = df[df.index >= "2000-01"]
plot_series(df, title="Polish inflation, from year 2000")

Już "na oko" widać tutaj ewidentne cykle i trendy. Natomiast są też dobre wieści - inflacja jest dość łagodnie zmiennym agregatem. Pytanie, co z sezonowością?

**Zadanie 1 (0.5 punktu)**

Uzupełnij kod funkcji `plot_stl_decomposition`. Wykorzystaj `STLTransformer` do obliczenia dekompozycji STL ([dokumentacja](https://www.sktime.net/en/v0.29.0/api_reference/auto_generated/sktime.transformations.series.detrend.STLTransformer.html)). Pamiętaj o podaniu odpowiednich opcji, aby uwzględnić podany okres sezonowości, i zwrócić wszystkie trzy komponenty.

Następnie narysuj dekompozycję STL dla danych o inflacji. Skomentuj:
- czy twoim zdaniem widać tutaj roczną sezonowość?
- czy rezydua są białym szumem, czy widać tam jeszcze jakieś informacje do wykorzystania?

In [ ]:
from sktime.transformations.series.detrend import STLTransformer


def plot_stl_decomposition(data: pd.Series, seasonal_period: int = 12) -> None:
    ...

    fig, axs = plt.subplots(4, figsize=(15, 10))

    # plot series, trend, seasonality and residuals in subplots
    ...


// skomentuj tutaj

Metoda "na oko" z dekompozycją STL jest ważna - to pozwala nabrać intuicji i wiedzy co do danych, i walidować parametry. Ale od tego są procedury automatyczne, z testami statystycznymi, żeby nie trzeba było tak ręcznie.

Sprawdźmy teraz, jak wygląda kwestia sezonowości i stacjonarności. Nie jest to stricte potrzebne dla modeli ETS - one przyjmują dane as-is. Natomiast dla modeli ARIMA jest to już niezbędne, bo wymagają danych stacjonarnych, a wiedza o sezonowości potrafi bardzo przyspieszyć obliczenia (SARIMA jest dużo wolniejsza).

**Zadanie 2 (0.75 punktu)**

1. Sprawdź za pomocą testów statystycznych dla sezonowości, czy w danych występuje sezonowość kwartalna, półroczna lub roczna. Przyda się funkcja `nsdiffs` z biblioteki pmdarima. Jeżeli wykryto sezonowość, to usuń ją z pomocą klasy `Differencer` z sktime ([dokumentacja](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.transformations.series.difference.Differencer.html)) oraz narysuj wartości na wykresie.

2. Sprawdź za pomocą testów statystycznych dla stacjonarności, jaki rząd różnicowania stacjonaryzuje szereg. Przyda się funkcja `ndiffs` z biblioteki pmdarima. Jeżeli jest większy niż zero, to usuń stacjonarość za pomocą klasy `Differencer` i narysuj wykres wartości po różnicowaniu.

3. Skomentuj, z jakiego wariantu modeli ARIMA byś skorzystał i dlaczego, na podstawie obecnej wiedzy: ARMA, ARIMA czy SARIMA.

Skorzystaj z domyślnych wartości `D_max` oraz `d_max`.

**Uwaga:** stwórz tutaj nowe zmienne dla wartości po różnicowaniu, nie nadpisuj `df`. Jeszcze nam się przyda.

// skomentuj tutaj

Jesteśmy już zasadniczo gotowi do treningu modeli. Do testowania wykorzystamy 20% najnowszych danych, ze strategią expanding window, i krokiem 1 (bo mamy odczyty inflacji co miesiąc). Naszymi metrykami będą MAE, SMAPE oraz MASE.

Wykonamy też analizę rezyduów. Błędy powinny mieć rozkład normalny (model bez biasu) oraz nie mieć autokorelacji (model wykorzystujący całą wiedzę). Na potrzeby wszystkich testów statystycznych zakładamy poziom istotności $\alpha = 0.05$.

Wykorzystamy test normalności Anderson-Darling, bo jest nieco "luźniejszy" niż test Shapiro-Wilk, co jest przydatne w praktyce - błędy rzadko kiedy są bardzo blisko rozkładu normalnego. Hipotezą zerową jest, że wartości pochodzą z danego rozkładu (domyślnie normalnego), a alternatywną, że z jakiegoś innego.

Dla testowania autokorelacji błędów użyjemy testu Ljung-Box, który testuje autokorelację błędów dla różnych lagów. Dla każdego laga ma osobny test, w którym hipotezą zerową jest brak autokorelacji, a hipotezą alternatywą autokorelacja błędów dla danego opóźnienia.

**Zadanie 3 (1.5 punktu)**

Uzupełnij kod funkcji `evaluate_model`:
1. Stwórz `ExpandingWindowSplitter` ([dokumentacja](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.split.ExpandingWindowSplitter.html)), który zacznie swoje działanie od 80% danych. Wielkość okna wyznacza argument `horizon`.
2. Stwórz listę metryk składającą się z MAE, SMAPE oraz MASE, z biblioteki sktime.
3. Przeprowadź ewaluację modelu za pomocą funkcji `evaluate` ([dokumentacja](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_evaluation.evaluate.html)). Przekaż `return_data=True`, żeby zwrócić też obliczone predykcje. Zwraca ona DataFrame z wynikami.
4. Oblicz średnie wartości metryk z wynikowego DataFrame'a. Wypisz je zaokrąglone do 2 miejsc po przecinku.
5. Uwzględniając argument `analyze_residuals`, przeprowadź analizę błędów:
   - oblicz rezydua jako $y - \hat{y}$
   - narysuj histogram rezyduów
   - wykonaj test Anderson-Darling z biblioteki `scipy` i wypisz, czy rozkład jest normalny, czy nie
   - przetestuj test Ljung-Box z biblioteki `statsmodels` i wypisz wyniki testu

Następnie przetestuj tę funkcję na dwóch baseline'ach: średniej oraz ostatniej znanej wartości. Skorzystaj z klasy `NaiveForecaster` ([dokumentacja](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.naive.NaiveForecaster.html)), i horyzontu czasowego 3 miesięcy. Narysuj też wykresy predykcji.

In [ ]:
from scipy.stats import anderson
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import (
    MeanAbsoluteScaledError,
    MeanAbsoluteError,
    MeanAbsolutePercentageError,
)
from sktime.forecasting.model_selection import ExpandingWindowSplitter
from statsmodels.stats.diagnostic import acorr_ljungbox


def evaluate_model(
    model,
    data: pd.Series,
    horizon: int = 1,
    plot_forecasts: bool = False,
    analyze_residuals: bool = False,
) -> None:
    cv = ...
    metrics = []
    results = ...

    # extract and print metrics
    mae = ...
    smape = ...
    mase = ...

    y_pred = pd.concat(results["y_pred"].values)

    if plot_forecasts:
        y_true = data[y_pred.index]
        plot_series(data, y_pred, labels=["y", "y_pred"])
        plt.show()
        plt.clf()
    
    if analyze_residuals:
        ...


Mamy już pierwsze baseline'y, wyniki z nich wyglądają dość solidnie. Zobaczmy, czy ETS albo ARIMA będą w stanie je przebić.

**Zadanie 4 (0.75 punktu)**

1. Dokonaj predykcji algorytmem AutoETS (klasa `StatsForecastAutoETS`), w wariancie damped trend. Narysuj też predykcje i dokonaj analizy błędów.
2. Analogicznie, dokonaj predykcji algorytmem AutoARIMA (klasa `StatsForecastAutoARIMA`). Jeżeli nie wykryto wcześniej sezonowości, to przekaż odpowiednią opcję, żeby ją wyłączyć - SARIMA jest dużo wolniejsza niż ARIMA.
3. Skomentuj wyniki:
   - czy udało się przebić baseline'y?
   - który z modeli jest lepszy, i o czym może to świadczyć?
   - czy modele są, przynajmniej w przybliżeniu, poprawne, tzn. mają normalne i nieskorelowane błędy?
   - czy wyniki najlepszego modelu są subiektywnie zadowalające?

Wykonaj tutaj predykcje 3-miesięczne, tak jak wcześniej.

// skomentuj tutaj

Prognozowanie 3-miesięczne było jednak dość krótkim horyzontem czasowym. Pytanie, co z dalszymi - półrocznym i rocznym. Są one często równie, albo nawet bardziej interesujące, np. w kontekście planowania budżetowego.

**Zadanie 5 (0.75 punktu)**

Dokonaj prognoz dla 6-miesięcznego oraz rocznego horyzontu:
- 2 baseline'ami
- metodami ETS oraz ARIMA
- dla najlepszego modelu narysuj prognozy oraz dokonaj analizy błędów

Skomentuj:
- czy widać jakieś zmiany między modelami w stosunku do przypadku 3-miesięcznego?
- jak zmienia się jakość prognoz przy większych horyzontach czasowych?
- czy te modele dla dłuższych horyzontów są twoim zdaniem użyteczne?

// skomentuj tutaj

## Prognozowanie ruchu sieciowego

Teraz zajmiemy się problemem o zupełnie innej charakterystyce - prognozowaniem ruchu sieciowego. Jest to kluczowe zadanie pod kątem skalowania serwerów, coraz częściej realizowane z pomocą właśnie prognozowania szeregów czasowych. Tzw. predictive scaling implementują między innymi [AWS](https://docs.aws.amazon.com/autoscaling/ec2/userguide/ec2-auto-scaling-predictive-scaling.html), [GCP](https://cloud.google.com/compute/docs/autoscaler/predictive-autoscaling) i [Azure](https://learn.microsoft.com/en-us/azure/azure-monitor/autoscale/autoscale-predictive), istnieją także rozwiązania dla Kubernetesa, [open source](https://predictive-horizontal-pod-autoscaler.readthedocs.io/en/latest/), oraz [komercyjne](https://keda.sh/blog/2022-02-09-predictkube-scaler/). Prognozowanie szeregów czasowych pozwala proaktywnie dokładać kolejne maszyny wirtualne, aby przygotować się na zwiększony ruch i zapewnić niskie opóźnienia, oraz redukuje koszty, automatycznie wyłączając je przy przewidywanym małym ruchu.

Wikipedia i Google hostowały [konkurs na platformie Kaggle](https://www.kaggle.com/c/web-traffic-time-series-forecasting), w którym zadaniem było prognozowanie ruchu na poszczególnych stronach. Jest on dość kolosalny, dlatego na jego podstawie przygotowano uproszczony, w którym mamy zapisaną sumaryczną liczbę requestów do Wikipedii, liczoną w milionach.

Charakterystyki takiego zadania to typowo:
- krótkoterminowe prognozy
- duża częstotliwość danych
- dynamicznie zmienne i zaszumione dane (np. przez aktywność botów)
- bardzo częsty retrening modeli
- duża potrzeba automatyzacji, brak ręcznej analizy modeli

In [ ]:
df = pd.read_parquet("wikipedia_traffic.parquet")
df = df.set_index("date").to_period(freq="d")
plot_series(df)
df

**Zadanie 6 (1 punkt)**

Zakładając horyzont 1 dnia, wytrenuj modele i dokonaj ich ewaluacji (analogicznie do poprzedniego zbioru danych):
- 2 modele baseline'owe
- model ETS (z damped trend)
- model ARIMA (bez sezonowości)
- model SARIMA

Skomentuj:
- czy z tych wyników można wnioskować, że mamy tu sezonowość?
- czy udało się przebić baseline?

Może jednak da się lepiej? Nasze dane mają bardzo dużą zmienność, a więc wariancję, a modele ARIMA tego nie lubią. Zastosujmy więc transformacje stabilizujące wariancję. Mamy tu same dodatnie wartości, więc nie będzie tu problemów numerycznych i można używać dowolnych operacji.

Trzeba tutaj wykorzystać `Pipeline` z sktime, bo automatycznie odwróci on wszystkie operacje podczas predykcji. Czasem dokonuje się ewaluacji danych po transformacji, ale typowo interesuje nas jakość na pierwotnych danych. Przekształcenia służą tylko do poprawienia treningu.

**Zadanie 7 (0.5 punktu)**

Stwórz pipeline, składający się z transformacji oraz modelu AutoARIMA (bez sezonowości). Wypróbuj transformacje:
- log
- sqrt
- Box-Cox

Przedstaw na wykresie oraz pierwotny szereg czasowy szereg po transformacji dającej najlepszy wynik.

Funkcję `make_pipeline` oraz klasy implementujące transformacje znajdziesz w sktime.

Skomentuj:
- czy udało się poprawić wynik transformacją?
- oceniając wizualnie, czy udało się ustabilizować wariancję?

## Prognozowanie sprzedaży

Najpowszechniejszym zastosowaniem prognozowania szeregów czasowych jest przewidywanie zapotrzebowania, sprzedaży, popytu, wydatków etc., czyli wszystkich typowych wskaźników dla przedsiębiorstwa. Musi to robić zasadniczo każda firma, dlatego nawet Excel oferuje rozbudowane możliwości prognozowania szeregów czasowych.

Zajmiemy się teraz kluczowym zadaniem dla włoskiej gospodarki, jakim jest prognozowanie sprzedaży makaronu. Zbiór danych został zebrany przez włoskich naukowców na potrzeby [tego artykułu naukowego](https://www.sciencedirect.com/science/article/abs/pii/S0957417421005431?via%3Dihub). Dane pochodzą z lat 2014-2018 (do końca roku), z 4 firm, i dotyczą wielu wyrobów z makaronu. Zawierają też dane o promocjach na poszczególne produkty. Niektórych danych brakuje, i wartości te trzeba imputować.

Dane tego typu mają typowo następujące cechy:
- często trend rosnący, mniejszy lub większy
- mocną sezonowość, często więcej niż jedną
- dużą czułość na powtarzalne okazje, np. weekendy czy święta
- duże i powtarzalne outliery
- dość duży szum i zmienność
- relatywnie niską częstotliwość, dzienną lub niższą
- często długie horyzonty dla prognozowania: miesięczne, kwartalne czy roczne
- zmienne egzogeniczne

**Zadanie 8 (1 punkt)**

1. Wczytaj dane z pliku `"italian_pasta.csv"`
2. Wybierz kolumny z firmy B1 (mają `"B1"` w nazwie) oraz kolumnę `"DATE"`
3. Stwórz kolumnę `"value"`:
   - sumaryczna sprzedaż makaronu
   - suma kolumn z `"QTY"` w nazwie
4. Stwórz kolumnę `"num_promos"`:
   - sumaryczna liczba aktualnych promocji
   - suma kolumn z `"PROMO"` w nazwie
5. Pozostaw tylko kolumny `"DATE"`, `"value"` oraz `"num_promos"`
6. Stwórz indeks typu datetime:
   - zmień typ kolumny `"DATE"` na datetime
   - ustaw tę kolumnę jako indeks
   - ustaw jej częstotliwość (frequency) jako dzienną, `"d"`
7. Podziel dane na:
   - zmienną `y`, `pd.Series` z kolumny `"value"`, główny szereg czasowy
   - zmienną `X`, `pd.Series` z kolumny `"num_promos"`, zmienne egzogeniczne
8. Uzupełnij wartości brakujące w zmiennych egzogenicznych zerami - możemy założyć, że domyślnie nie ma promocji.
9. Przedstaw na wykresie szereg czasowy `y`. Pamiętaj o podaniu tytułu wykresu.

Tym razem będzie nas interesowało prognozowanie w długim horyzoncie czasowym. Zakładamy, że nasz klient, włoski producent makaronu, ma dane z lat 2014-2017, i chce prognozować swoją sprzedaż w roku 2018. Takie prognozy są potrzebne np. do zakontraktowania długoterminowych dostaw i produkcji na kolejny rok. Z naszej perspektywy jest to trudne, ale przynajmniej szybkie, bo mamy tylko jeden zbiór treningowy i testowy, a nie serię, jak w expanding window.

Do ewaluacji posłuży nam teraz funkcja `evaluate_pasta_sales_model`.

**Zadanie 9 (1 punkt)**

Uzupełnij kod funkcji do ewaluacji:
- podziel `y` na zbiór treningowy i testowy według daty, wszystko od `2018-01-01` włącznie to zbiór testowy
- jeżeli przekazano `X`, to je tak samo podziel
- dokonaj imputacji wartości brakujących w `y`:
  - klasa `Imputer` z sktime
  - zastosuj strategię `ffill` (uzupełnienie ostatnią znaną wartością)
- wytrenuj model (pamiętaj o przekazaniu `X`), dokonaj predykcji
- dokonaj ewaluacji predykcji za pomocą MAE, SMAPE oraz MASE (znajdź odpowiednie funkcje w sktime)
- wypisz wyniki do 2 miejsca po przecinku
- skopiuj kod dla `plot_forecasts` i `analyze_residuals` z zadania 3

In [ ]:
from typing import Optional

import numpy as np
from sktime.transformations.series.impute import Imputer


def evaluate_pasta_sales_model(
    model,
    df: pd.Series,
    X: Optional[np.ndarray] = None,
    plot_forecasts: bool = False,
    analyze_residuals: bool = False,
) -> None:
    # split data

    # impute values

    # fit, predict, calculate and print metrics

    # plot_forecasts

    # analyze_residuals


Teraz nie pozostaje nam nic, tylko prognozować.

**Zadanie 10 (1.5 punktu)**

Dokonaj predykcji modelami:
- 2 baseline'ami
- ETS (z damped trend)
- ARIMA
- SARIMA z sezonowością 30 dni
- ARIMAX
- SARIMAX z sezonowością 30 dni

Dla najlepszego modelu wypróbuj transformacje log, sqrt oraz Box-Cox.

Na koniec narysuj predykcje oraz dokonaj analizy błędów dla finalnego modelu.

Skomentuj:
- czy udało się przebić baseline?
- czy finalny model uwzględnia sezonowość i/lub zmienne egzogeniczne (dane o promocjach)?
- czy warto było stosować transformacje danych?
- skomentuj ogólne zachowanie modelu na zbiorze testowym, na podstawie wykresu predykcji
- czy model jest nieobciążony (rozkład normalny błędów o średniej 0), bez autokorelacji, czy jest tutaj miejsce na poprawę?

Zmienne egzogeniczne można jeszcze rozbudować - przykładowo, zachowanie kupujących w weekendy i święta jest inne. Typowo sprzedaż mocno rośnie przed i po okresach, kiedy sklepy są zamknięte, no i naturalnie spada do zera, kiedy sklep jest nieczynny.

**Zadanie 11 (0.75 punktu)**

1. Stwórz listę zmiennych reprezentujących święta z pomocą klasy `HolidayFeatures`:
   - przyda się funkcja `country_holidays` z biblioteki `holidays`
   - pamiętaj o tym, że operujemy we Włoszech, skrót kraju `"IT"`
   - uwzględnij weekendy
   - stwórz po prostu jedną zmienną "czy jest święto" (opcje `return_dummies` i `return_indicator`)
2. Dodaj te zmienne do naszych danych o promocjach `X`. Przyda się `pd.merge` oraz opcje `left_index` i `right_index`.
3. Wytrenuj model ARIMAX (lub SARIMAX, jeżeli wcześniej wykryto sezonowość). Wykorzystaj najlepszą transformację z poprzedniego zadania.
4. Skomentuj, jak mają się jego wyniki do wcześniejszych.

// skomentuj tutaj

## Zadanie dodatkowe (3 punkty)

Klasyfikacja szeregów czasowych często wykorzystuje dane z akcelerometrów, smartfonów, smartwatchy i innych tego typu urządzeń. Praktycznie każdy wyżej klasy smartwatch ma wbudowane proste klasyfikatory analizujące aktywność, tętno, ruch itp. Ciekawym zastosowaniem miary takiej aktywności dobowej jest diagnostyka medyczna, bo niektóre choroby powodują nieoczywiste na pierwszy rzut oka zmniejszenie motoryki człowieka.

[Zbiór danych Depresjon](https://datasets.simula.no/depresjon/) dotyczy diagnozy depresji na podstawie aktywności dobowej mierzonej smartwatchem ActiGraph. Zbiór ze względu na wrażliwą naturę problemu jest bardzo mały, 55 pacjentów, ale trzeba sobie z tym radzić, bo w medycynie jest to dość normalne. Pomiary per pacjent są dość długie, około 2 tygodni.

1. Wczytaj dane, upewnij się że mają prawidłowe typy
2. Uzupełnij wartości brakujące per pacjent za pomocą interpolacji liniowej (klasa `Imputer`)
3. Do ewaluacji wykorzystaj podejście z [tego artykułu](https://www.cai.sk/ojs/index.php/cai/article/view/2021_4_850), tzw. nested cross-validation:
   - 5-fold CV (stratified) do testowania (outer)
   - LOOCV do wyboru hiperparametrów (inner)
4. Dokonaj ekstrakcji cech algorytmami:
   - [MiniROCKET](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.transformations.panel.rocket.MiniRocket.html)
   - [Shapelet transform](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.transformations.panel.shapelet_transform.ShapeletTransform.html)
   - [TSFRESH](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.transformations.panel.tsfresh.TSFreshRelevantFeatureExtractor.html) (w wariancie "efficient")
   - [Catch22](https://www.sktime.net/en/latest/api_reference/auto_generated/sktime.transformations.panel.catch22.Catch22.html)
5. Wykorzystaj regresję logistyczną (`LogisticRegressionCV`) jako klasyfikator
6. Dokonaj ewaluacji za pomocą metryk:
   - balanced accuracy
   - F1
   - precision
   - recall
   - specificity
   - AUROC
   - MCC (Matthews Correlation Coefficient)
7. Porównaj wyniki z [artykułem](https://www.cai.sk/ojs/index.php/cai/article/view/2021_4_850) (tabela 4). Skomentuj, czy udało się przebić te wyniki.